# Pristup stranici gdje su artikli i skidanje pdfova u folder

In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 17.5 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import os
import requests
import tempfile

Prijava na stranicu radi mogućnosti skidanja pdf-a

In [ ]:
DOWNLOAD_FOLDER = "pdfovi"
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

In [ ]:
# ➤ Tvoje korisničko ime i lozinka za CEEOL
USERNAME = "akazazovic1@etf.unsa.ba"
PASSWORD = "kakoTako67"

def web_driver():
    options = webdriver.ChromeOptions()

    # Postavke za Chrome WebDriver
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')  # Pokreće Chrome bez UI
    options.add_argument('--disable-gpu')  # Za headless režim
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')  # Povećava stabilnost
    options.add_experimental_option("prefs", {
        "download.default_directory": os.path.abspath(DOWNLOAD_FOLDER),  # Postavi direktorij za preuzimanje
        "download.prompt_for_download": False,  # Ne prikazuje prompt za preuzimanje
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True  # Omogućava automatsko preuzimanje
    })
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()
wait = WebDriverWait(driver, 15)

driver.get("https://www.ceeol.com/search/journal-detail?id=2892")
driver.maximize_window()

login_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li[data-ng-click='openlogInDlg()']")))
login_btn.click()
time.sleep(5)
#wait.until(EC.visibility_of_element_located((By.ID, "loginUserName")))
driver.find_element(By.ID, "loginUserName").send_keys(USERNAME)
driver.find_element(By.NAME, "password").send_keys(PASSWORD)
login_modal_btn = driver.find_element(By.XPATH, "//button[@ng-click='logIn()']")
login_modal_btn.click()
time.sleep(5)

#vracanje na dio gdje je clanak
driver.get("https://www.ceeol.com/search/journal-detail?id=2892")
driver.maximize_window()

Skidanje linkova za sve pdfove

In [ ]:
# Dohvati sve članke
articles = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-ng-repeat="article in articles"]')))

print(f"Pronađeno članaka: {len(articles)}")

pdf_links = []

for article in articles:
    try:
        # Pronađi link za preuzimanje PDF-a
        pdf_link_elem = article.find_element(By.CSS_SELECTOR, 'a.fa-file-pdf-o')

        # Dobij link za preuzimanje PDF-a
        pdf_url = pdf_link_elem.get_attribute("data-ng-href")

        # Ako je link relativan, dodaj osnovnu URL adresu
        if pdf_url.startswith('/'):
            pdf_url = "https://www.ceeol.com" + pdf_url

        # Dodaj PDF link u listu
        pdf_links.append(pdf_url)

    except Exception as e:
        print(f"Greška pri dohvaćanju PDF linka: {e}")
        continue

print(f"Ukupno PDF linkova: {len(pdf_links)}")


for pdf_url in pdf_links:
    try:
        print(f"Preuzimam PDF: {pdf_url}")
        driver.get(pdf_url)
        time.sleep(3)

        download_button = wait.until(
            EC.presence_of_element_located((By.ID, 'downloadButton'))
        )
        download_button.click()
        time.sleep(4)

        driver.get("https://www.ceeol.com/search/journal-detail?id=2892")

    except Exception as e:
        print(f"Greška kod preuzimanja PDF-a: {e}")
        continue


Pronađeno članaka: 37
Ukupno PDF linkova: 37
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136733
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136738
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136739
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136740
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136741
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136746
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136751
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136752
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136753
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.com/search/viewpdf?id=1136754
[ PDF preuzimanje započeto.]
Preuzimam PDF: https://www.ceeol.co

Prebacivanje foldera s pdfovima na drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

source_folder = '/content/pdfovi'
destination_folder = '/content/drive/My Drive/Kazazovic_Amina/pdfovi'

# provjera imal taj folder
if not os.path.exists(destination_folder):
    shutil.move(source_folder, destination_folder)
    print(f"Folder sa PDF-ovima uspešno premješten na: {destination_folder}")
else:
    print(f"Folder već postoji na odredištu: {destination_folder}. Premještanje nije izvršeno.")

Folder već postoji na odredištu: /content/drive/My Drive/Kazazovic_Amina/pdfovi. Premještanje nije izvršeno.


# Rad s pojedinačnim pdfovima

In [4]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 67.7 MB/s eta 0:00:00


In [ ]:
import pdfplumber
pdf_folder="/content/drive/My Drive/Kazazovic_Amina/pdfovi"

def process_pdf(pdf_path):
    print(f"Obrađujem PDF: {pdf_path}")

    with pdfplumber.open(pdf_path) as pdf:
        print(f"Broj stranica u PDF-u: {len(pdf.pages)}")

for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        process_pdf(pdf_path)

Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/ancient-history-of-today-s-herzegovina-in-the-work-of-arthur-evans_Content File-PDF.pdf


Broj stranica u PDF-u: 13
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/bi%c5%a1%c4%87e-and-blagaj-in-the-deliberations-of-the-ragusan-councils-in-the-15th-century-excerpts-from-archival-sources_Content File-PDF.pdf


Broj stranica u PDF-u: 42
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/the-role-of-guest-radin-butkovi%c4%87-in-the-construction-of-fortifications-on-the-site-of-the-old-bridge-with-reference-to-the-beginning-of-the-urbanization-of-mostar_Content File-PDF.pdf


Broj stranica u PDF-u: 29
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/ste%c4%87aks-in-the-inner-city-center-of-mostar-with-a-focus-on-preservation-and-protection_Content File-PDF.pdf


Broj stranica u PDF-u: 33
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/the-space-of-herzegovina-through-the-prism-of-early-ottoman-chronicles_Content File-PDF.pdf


...

Broj stranica u PDF-u: 22
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/linguo-stylistic-procedures-in-the-literary-work-sa-plo%c4%8da-isto%c4%8dnih-by-hamza-humo_Content File-PDF.pdf


Broj stranica u PDF-u: 14
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/expressive-means-in-the-narratives-of-hamza-humo_Content File-PDF.pdf


Broj stranica u PDF-u: 18
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/works-of-famous-mostar-writers-in-the-primary-education-system-of-bosnia-and-herzegovina-representation-of-mostar-writers-in-curricula-in-bosnia-and-herzegovina_Content File-PDF.pdf


Broj stranica u PDF-u: 14
Obrađujem PDF: /content/drive/My Drive/Kazazovic_Amina/pdfovi/blue-red-and-green-interactions-in-the-mostar-s-reality_Content File-PDF.pdf


Broj stranica u PDF-u: 19


In [5]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 41.8 MB/s eta 0:00:00


In [6]:
pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=6fa6cb5572ba84b365dbdb10352f13ae8efd7ebdc32cbbd60268e1e9006d5923
  Stored in directory: /root/.cache/pip/wheels/32/6a/b6/b7eb43a6ad55b139c15c5daa29f3707659cfa6944d3c696f5b
Successfully built langid


In [7]:
pip install transliterate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.1 MB/s eta 0:00:00


In [8]:
import pdfplumber
import fitz
import re
import langid
from transliterate import translit

redni_broj=1

def ukloni_citirane_reference(text):
    pattern = r"\((?:[A-ZČĆŽŠĐa-zčćžšđ]+\s)?\d{4}:?\s*(?:[^)]*?)?\s*(?:\d+)?\)"
    return re.sub(pattern, "", text).strip()

def ukloni_broj_broj_reference(text):
    pattern = r"\([^:]*\d+:\s*\d+[^)]*\)"
    return re.sub(pattern, "", text).strip()

def ukloni_broj_razmak_dvotacka_broj(text):
    pattern = r"\([^)]*\d+\s*:\s*\d+[^)]*\)"
    return re.sub(pattern, "", text).strip()

def ukloni_format_hha(text):
  pattern = r"\([A-ZČĆŽŠĐ]+\:\d+\)"
  return re.sub(pattern, "", text).strip()

def ukloni_slovo_tacka_broj(text):
  pattern = r"\b[A-ZČĆŽŠĐa-zčćžšđ]\.\d+\b"
  return re.sub(pattern, "", text).strip()

def extract_pdf_metadata(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text("text")+"\n"

    #NOVINA
    novina_match = re.search(r"Source:\s*(.+)", full_text)
    novina = novina_match.group(1).strip() if novina_match else "N/A"

    #DATUM
    datum_match = re.search(r"Issue:\s*([\d/]+)", full_text)
    datum = datum_match.group(1).strip() if datum_match else "N/A"

    #RUBRIKA
    rubrika = "N/A"

    #NADNASLOV
    nadnaslov = "N/A"

    #NASLOV
    naslov_match = re.search(r"(?:Citation style:\s*)?\"([^\"]*)\"", full_text)
    if naslov_match:
        naslov = naslov_match.group(1).strip().replace('\n', '')
    else:
        naslov = "N/A"

    #PODNASLOV
    podnaslov = "N/A"

    #STRANA
    strana_matches = re.findall(r"SLOVO O\s+MOSTARU\s*\n\s*(\d+)", full_text, re.IGNORECASE)
    if strana_matches:
        strana = f"{strana_matches[0]}-{str(int(strana_matches[-1]) + 1)}"
    else:
        strana = "N/A"

    #AUTOR(I)
    autor_match = re.search(r"Author\(s\):\s*(.+)", full_text)
    autor = autor_match.group(1).strip() if autor_match else "N/A"

    #uklanjanje svega do KLJUČNE RIJEČI
    ključne_riječi_match = re.search(r"(Ključne riječi)", full_text, re.IGNORECASE)
    if ključne_riječi_match:
        index_ključne_riječi = ključne_riječi_match.start()
        article_body = full_text[index_ključne_riječi:].strip()
    else:
        article_body = full_text.strip()

    #uklanjanje KLJUČNE RIJEČI i nabrojanih riječi na osnovu pdfa
    if redni_broj==1 or redni_broj==23:
      broj_redova_za_uklanjanje=8
    elif redni_broj==2 or redni_broj==17 or redni_broj==37 or redni_broj==31 or redni_broj==33 or redni_broj==24 or redni_broj==25 or redni_broj==22 or redni_broj==20 or redni_broj==3 or redni_broj==8 or redni_broj==11 or redni_broj==12:
      broj_redova_za_uklanjanje=2
    elif redni_broj==4 or redni_broj==5 or redni_broj==7:
      broj_redova_za_uklanjanje=6
    elif redni_broj==6 or redni_broj==9 or redni_broj==30 or redni_broj==29 or redni_broj==18 or redni_broj==13 or redni_broj==14 or redni_broj==15:
      broj_redova_za_uklanjanje=3
    elif redni_broj==10 or redni_broj==34:
      broj_redova_za_uklanjanje=7
    elif redni_broj==32:
      broj_redova_za_uklanjanje=0
    elif redni_broj==19 or redni_broj==36:
      broj_redova_za_uklanjanje=4
    elif redni_broj==21:
      broj_redova_za_uklanjanje=13
    elif redni_broj==26 or redni_broj==28:
      broj_redova_za_uklanjanje=9
    elif redni_broj==27:
      broj_redova_za_uklanjanje=1
    elif redni_broj==35:
      broj_redova_za_uklanjanje=5
    elif redni_broj==16:
      broj_redova_za_uklanjanje=-1

    if broj_redova_za_uklanjanje > 0:
     pattern = r"(Ključne riječi)\s*:\s*" + r"(?:.*\n){" + str(broj_redova_za_uklanjanje) + "}"
     article_body = re.sub(pattern, "", article_body, flags=re.IGNORECASE).strip()
    elif broj_redova_za_uklanjanje==0:
      pattern_ukloni_do_uvod = r"^(.*?)(?:UVOD)"
      article_body = re.sub(pattern_ukloni_do_uvod, "", article_body,flags=re.IGNORECASE | re.DOTALL).strip()
    elif broj_redova_za_uklanjanje==-1:
      pattern_ukloni_do_drugo_uvod = r"^(?:.*?(?:UVOD)){1}(.*?)(?:UVOD)"
      article_body = re.sub(pattern_ukloni_do_drugo_uvod, "", article_body, flags=re.IGNORECASE | re.DOTALL).strip()

    article_body = re.sub(r"\b(ZAKLJUČAK)\b", "", article_body, flags=re.IGNORECASE).strip()
    #uklanjanje svega nakon rijeci summary
    article_body = re.split(r"\bSummary\b", article_body, flags=re.IGNORECASE)[0].strip()

    #uklanjanje literature i sve do dole
    pattern_ukloni_od_literature = r"(Literatura.*?)$"
    article_body = re.sub(pattern_ukloni_od_literature, "", article_body, flags=re.DOTALL).strip()

    #uklanjanje sve nakon izvori i literatura
    pattern_ukloni_od_izvori_literatura = r"(Izvori i literatura.*?)$"
    article_body = re.sub(pattern_ukloni_od_izvori_literatura, "", article_body, flags=re.DOTALL).strip()

    #uklanjaje conclusion i sve do dole
    pattern_ukloni_od_conclusion = r"(CONCLUSION.*?)$"
    article_body = re.sub(pattern_ukloni_od_conclusion, "", article_body, flags=re.IGNORECASE | re.DOTALL).strip()


    article_body = re.sub(r"\n+", " ", article_body)  # novi redovi
    article_body = re.sub(r"EEOL copyright 2025 ZBORNIK RADOVA \d+", "", article_body, flags=re.IGNORECASE).strip()
    article_body = re.sub(r"EEOL copyright 2025 SLOVO O MOSTARU \d+", "", article_body, flags=re.IGNORECASE).strip()
    #izbacivanje riječi UVOD i 1. UVOD
    article_body = re.sub(r"UVOD ", "", article_body).strip()
    article_body = re.sub(r"1. UVOD ", "", article_body).strip()
    article_body = re.sub(r"\s{2,}", " ", article_body)  # višestruki razmaci

    article_body=ukloni_citirane_reference(article_body)
    article_body=ukloni_broj_broj_reference(article_body)
    article_body=ukloni_broj_razmak_dvotacka_broj(article_body)
    article_body=ukloni_format_hha(article_body)
    article_body=ukloni_slovo_tacka_broj(article_body)
    article_body=article_body.replace(" ,",",")
    article_body=article_body.replace(" .",".")
    article_body=article_body.replace("CEEOL copyright 2025 C","")
    article_body=article_body.replace("***"," ")
    article_body=article_body.replace("  "," ")

    #print(article_body)

    formatted = "<***>\n"
    formatted += f"NOVINA: {novina}\n"
    formatted += f"DATUM: {datum}\n"
    formatted += f"RUBRIKA: {rubrika}\n"
    formatted += f"NADNASLOV: {nadnaslov}\n"
    formatted += f"NASLOV: {naslov}\n"
    formatted += f"PODNASLOV: {podnaslov}\n"
    formatted += f"STRANA: {strana}\n"
    formatted += f"AUTOR(I): {autor}\n\n"
    formatted += article_body + "\n"

    return formatted

pdf_folder="/content/drive/My Drive/Kazazovic_Amina/pdfovi"
file_path = "/content/sviPdfovi.txt"
filenames = sorted(os.listdir(pdf_folder))

brojac=1
with open(file_path, 'w', encoding='utf-8') as file:
    for filename in filenames:
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            article_data = extract_pdf_metadata(pdf_path)
            redni_broj+=1
            file.write(article_data + "\n\n")

# TOKENIZACIJA


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

def prebroji_tokene_po_clanku(putanja_do_fajla):
    sav_tekst = ""
    with open(putanja_do_fajla, 'r', encoding='utf-8') as f:
        sav_tekst = f.read()

    clanci = sav_tekst.split("<***>")
    ukupno_svih_tokena = 0

    for i, clanak in enumerate(clanci):
        linije = clanak.strip().split('\n')
        pocetak_sadrzaja = -1
        broj_tokena_clanka = 0
        if i==0:
          continue

        for j, linija in enumerate(linije):
            if linija.startswith("AUTOR(I):"):
                if j + 2 < len(linije):
                    pocetak_sadrzaja = j + 2
                    break

        if pocetak_sadrzaja != -1:
            sadrzaj_clanka = "\n".join(linije[pocetak_sadrzaja:]).strip()
            if sadrzaj_clanka:
                tokenizer = Tokenizer()
                tokenizer.fit_on_texts([sadrzaj_clanka])
                tokens = tokenizer.texts_to_sequences([sadrzaj_clanka])
                broj_tokena_clanka = sum(len(sekvenca) for sekvenca in tokens)
                ukupno_svih_tokena += broj_tokena_clanka
                print(f"Broj tokena za članak {i}: {broj_tokena_clanka}")

    print(f"\nUkupan broj tokena u sadržaju svih članaka: {ukupno_svih_tokena}")

putanja_do_svih_pdfova = "/content/drive/My Drive/Kazazovic_Amina/sviPdfovi.txt"

# Prebrojite tokene po članku
prebroji_tokene_po_clanku(putanja_do_svih_pdfova)

Broj tokena za članak 1: 4322
Broj tokena za članak 2: 1754
Broj tokena za članak 3: 1861
Broj tokena za članak 4: 2673
Broj tokena za članak 5: 6243
Broj tokena za članak 6: 3240
Broj tokena za članak 7: 3530
Broj tokena za članak 8: 6039
Broj tokena za članak 9: 5362
Broj tokena za članak 10: 11349
Broj tokena za članak 11: 5859
Broj tokena za članak 12: 1809
Broj tokena za članak 13: 3011
Broj tokena za članak 14: 4152
Broj tokena za članak 15: 2925
Broj tokena za članak 16: 3979
Broj tokena za članak 17: 3692
Broj tokena za članak 18: 1537
Broj tokena za članak 19: 2565
Broj tokena za članak 20: 3176
Broj tokena za članak 21: 3556
Broj tokena za članak 22: 4442
Broj tokena za članak 23: 1504
Broj tokena za članak 24: 4711
Broj tokena za članak 25: 4295
Broj tokena za članak 26: 1995
Broj tokena za članak 27: 1572
Broj tokena za članak 28: 4079
Broj tokena za članak 29: 3644
Broj tokena za članak 30: 3535
Broj tokena za članak 31: 2408
Broj tokena za članak 32: 4948
Broj tokena za č